In [90]:
import torch
import torchvision
import pandas as pd
import numpy as np
import glob
import os
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
import sys
sys.path.append('../scripts/')
from torch_bad_ones import find_bad_ones_torch
from torch.utils.data import Dataset, DataLoader
import PIL
import collections
import math
from torch.nn import Module, Sequential
from torch.optim import Adam
from torchsummary import summary
import time
import cv2

In [2]:
train_files = glob.glob('../../ml-data-training/ship_segmentation_data/train_v2/*.jpg')

In [3]:
print(len(train_files))

192556


In [4]:
train_csv = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')

In [5]:
train_csv.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [6]:
updated_train_csv = train_csv.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()

In [7]:
def create_classification_labels(data):
    try:
        if math.isnan(data[0]) == True:
            return 0
    except:
        return 1
    # else:
    #     return 1

In [8]:
updated_train_csv['class_labels'] = updated_train_csv['EncodedPixels'].progress_apply(create_classification_labels)

100%|██████████| 192556/192556 [00:00<00:00, 283647.83it/s]


In [9]:
updated_train_csv

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0
...,...,...,...
192551,fffedbb6b.jpg,[nan],0
192552,ffff2aa57.jpg,[nan],0
192553,ffff6e525.jpg,[nan],0
192554,ffffc50b4.jpg,[nan],0


In [10]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels
0,00003e153.jpg,[nan],0
1,0001124c7.jpg,[nan],0
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1
4,0001b1832.jpg,[nan],0


In [11]:
updated_train_csv['fixed_paths'] = updated_train_csv['ImageId'].progress_apply(lambda x: f"../../ml-data-training/ship_segmentation_data/train_v2/" + x)

100%|██████████| 192556/192556 [00:00<00:00, 311342.65it/s]


In [12]:
updated_train_csv.head()

,ImageId,EncodedPixels,class_labels,fixed_paths
0,00003e153.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
1,0001124c7.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...
2,000155de5.jpg,[264661 17 265429 33 266197 33 266965 33 26773...,1,../../ml-data-training/ship_segmentation_data/...
3,000194a2d.jpg,[360486 1 361252 4 362019 5 362785 8 363552 10...,1,../../ml-data-training/ship_segmentation_data/...
4,0001b1832.jpg,[nan],0,../../ml-data-training/ship_segmentation_data/...


In [13]:
for x in updated_train_csv['fixed_paths'].values.tolist():
    if os.path.exists(x) == False:
        print(x)

In [14]:
to_check = updated_train_csv['fixed_paths'].values.tolist()

In [15]:
issues = find_bad_ones_torch(to_check)

In [16]:
updated_train_csv = updated_train_csv[updated_train_csv.fixed_paths != issues[0]]

In [17]:
def split_datasets(data, test_size = 0.01):
    train, test = train_test_split(data, test_size = test_size, random_state = 42) 
    train, val = train_test_split(train, test_size = test_size, random_state = 42)
    return train, val, test

In [18]:
train, val, test = split_datasets(updated_train_csv)

In [25]:
class ShipDataset(Dataset):
    def __init__(self, csv_file, return_label = True):
        self.csv_file = csv_file
        self.img_paths = self.csv_file['fixed_paths'].values.tolist()
        self.labels = self.csv_file['class_labels'].values.tolist()
        self.return_label = return_label

    def __getitem__(self, idx):
        img = PIL.Image.open(self.img_paths[idx])
        img = img.resize((256, 256)) 
        img = np.array(img)
        img = img / 255
        # numpy format: H X W X C
        # torch format: C X H X W
        img = img.transpose((2, 0, 1))
        label = self.labels[idx]
        img_tensor = torch.Tensor(img)
        label_tensor = torch.Tensor([label])
        if self.return_label:
            return tuple((img_tensor, label_tensor))
        else:
            return tuple((img_tensor))
        
    def __len__(self):
        return len(self.img_paths)

In [202]:
class ShipDataset(Dataset):
    def __init__(self, csv_file, return_label = True):
        self.csv_file = csv_file
        self.img_paths = self.csv_file['fixed_paths'].values.tolist()
        self.labels = self.csv_file['class_labels'].values.tolist()
        self.return_label = return_label

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.img_paths[idx])
        img = torchvision.io.decode_image(img)
        img = torchvision.transforms.Resize((256, 256))(img)
        img = img / 255
        # numpy format: H X W X C
        # torch format: C X H X W
        img_tensor = img
        label = self.labels[idx]
        label_tensor = torch.Tensor([label])
        if self.return_label:
            return tuple((img_tensor, label_tensor))
        else:
            return tuple((img_tensor))
        
    def __len__(self):
        return len(self.img_paths)

In [208]:
train_dataset = ShipDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=256, num_workers=20, persistent_workers=True)
val_dataset = ShipDataset(val, return_label = False)
val_dataloader = DataLoader(val_dataset, batch_size=256, num_workers=20, persistent_workers=True)

In [209]:
temp = None
for x, y in train_dataloader:
    temp = x
    break

In [234]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            torch.nn.Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
        )

        self.linear_layers = Sequential(
            torch.nn.Linear(16384, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.cnn_layers(x)
        # You can use flatten or this below line....
        # x = x.view(x.size(0), -1)
        x = self.linear_layers(x) 
        return x

In [235]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [236]:
model = Net()
optimizer = Adam(model.parameters(), lr=0.07)
criterion = torch.nn.BCELoss()
if torch.cuda.is_available():
    model = model.to(device)
    criterion = criterion.to(device)
# print(model)

In [237]:
train_dataloader.num_workers

20

In [74]:
n_epochs = 200
loss_fn = torch.nn.BCELoss()
model.train(True)
loss_vals = []
for epoch in tqdm(range(n_epochs)):
    for X_batch, y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_vals.append(loss)
    if epoch % 2 == 0:
        print(loss_vals[-1])
    # break

  0%|          | 1/200 [02:39<8:48:33, 159.37s/it]

tensor(0.5749, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


  2%|▏         | 3/200 [07:45<8:26:23, 154.23s/it]

tensor(0.5269, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


  2%|▎         | 5/200 [12:52<8:19:54, 153.82s/it]

...

 98%|█████████▊| 197/200 [8:23:57<07:40, 153.66s/it]

tensor(0.4281, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


100%|█████████▉| 199/200 [8:29:03<02:33, 153.50s/it]

tensor(0.5758, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


100%|██████████| 200/200 [8:31:36<00:00, 153.48s/it]


In [84]:
time.time()

1679583052.1654525

In [85]:
start = time.time()
n_epochs = 1
loss_fn = torch.nn.BCELoss()
model.train(True)
loss_vals = []
for epoch in tqdm(range(n_epochs)):
    for X_batch, y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_vals.append(loss)
    print(loss_vals[-1])
    # break
end = time.time()
print(end-start)

100%|██████████| 1/1 [02:58<00:00, 178.41s/it]

tensor(0.6158, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
178.42296648025513


In [238]:
start = time.time()
n_epochs = 1
loss_fn = torch.nn.BCELoss()
model.train(True)
loss_vals = []
for epoch in tqdm(range(n_epochs)):
    for X_batch, y_batch in train_dataloader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_vals.append(loss)
    print(loss_vals[-1])
    # break
end = time.time()
print(end-start)

100%|██████████| 1/1 [02:08<00:00, 129.00s/it]

tensor(26., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
129.0052387714386
